In [1]:
#!/usr/bin/env python
import os
import unittest
from unittest import TestCase
from cogent import LoadTable

def bonferroni_correction(table, alpha):
    """apply bonferroni correction to summary.txt file, the p-adjusted value is calculated as 0.05/n, where n is 
    number of total tests"""
    
    # this should be the sequential Bonferroni method -- CHECK!!
    sig_results = []
    num_rows = table.Shape[0]
    prob_index = table.Header.index('prob')
    table = table.sorted(columns='prob')
    
    #compare the smallest probability p_1 to p_adjusted (alpha / n). If p is greater than p_adjusted, declare 
    #all tests not significant; if p is samller than p_adjusted, declare this test is significant, 
    #and compare the second smallest p_2...
    
    for row in table.getRawData():
        p_adjusted = alpha / (num_rows - len(sig_results))
        if row[prob_index] > p_adjusted:
            break
        sig_results.append(row)
    
    if not sig_results:
        return None
    
    sub_table = LoadTable(header=table.Header, rows=sig_results)
    
    
    return sub_table

class TestBonferroni(TestCase):
    table_data = LoadTable('test_bonferroni.txt', sep='\t')
    alpha = 0.05
    
    def test_bonferroni(self):
        """an example table is constructed according to BIOMETRY 3rd edition, pp241, with known
        probs and significant results; raise the AssertionError when significant results 
        are not correctly produces after Bonferroni correction"""
        bon_adj_tbl = bonferroni_correction(self.table_data, self.alpha)
        sig_results = bon_adj_tbl.getRawData('prob')
        expect_results = [1e-06, 1e-06, 0.000002, 0.004765]
        self.assertEqual(sig_results, expect_results)

unittest.TextTestRunner(verbosity=2).run(TestBonferroni('test_bonferroni'))

seq_classes = ['ENU_variants/autosomes', 'ENU_variants/sex_chroms', 'ENU_variants/long_flanks/autosomes', 'ENU_variants/long_flanks/sex_chroms', 
               'ENU_vs_germline/autosomes', 'ENU_vs_germline/sex_chroms', 
               'germline_variants/autosomes', 'germline_variants/sex_chroms', 'germline_variants/long_flanks/autosomes', 'germline_variants/long_flanks/sex_chroms', 
               'ENU_A_vs_X', 'germline_A_vs_X']


summary_paths = []
for seq_class in seq_classes:
    seq_paths = !find ../results/$seq_class/* -name "summary.txt"
    summary_paths.append(seq_paths)

for paths in summary_paths:
    output_dir = os.path.commonprefix(paths)
    output_filename = output_dir + "bonferroni.txt"
    
    header = ['directions', 'positions', 'probs']
    rows = []
    for path in paths:
        direction = path.split('/')[-2]
        table = LoadTable(path, sep='\t')
        alpha = 0.05
        adjusted_tbl = bonferroni_correction(table, alpha)

        if adjusted_tbl is None:
            print "all tests in %s are not significant" % path
            continue

        adjusted_tbl_ncol = adjusted_tbl.withNewColumn('direction', lambda x: direction, columns=adjusted_tbl.Header[0])
        
        for row in adjusted_tbl_ncol.getRawData(['direction', 'Position', 'prob']):
            rows.append(row)
            
    summary_tbl = LoadTable(header=header, rows=rows, sep='\t')
    summary_tbl.writeToFile(output_filename, sep='\t')
    print "bonferroni corrected table %s is saved" % output_filename


test_bonferroni (__main__.TestBonferroni)
an example table is constructed according to BIOMETRY 3rd edition, pp241, with known ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


bonferroni corrected table ../results/ENU_variants/autosomes/directions/bonferroni.txt is saved
all tests in ../results/ENU_variants/sex_chroms/directions/TtoG/summary.txt are not significant
all tests in ../results/ENU_variants/sex_chroms/directions/AtoC/summary.txt are not significant
bonferroni corrected table ../results/ENU_variants/sex_chroms/directions/bonferroni.txt is saved
bonferroni corrected table ../results/ENU_variants/long_flanks/autosomes/directions/bonferroni.txt is saved
all tests in ../results/ENU_variants/long_flanks/sex_chroms/directions/AtoC/summary.txt are not significant
all tests in ../results/ENU_variants/long_flanks/sex_chroms/directions/GtoT/summary.txt are not significant
all tests in ../results/ENU_variants/long_flanks/sex_chroms/directions/CtoA/summary.txt are not significant
all tests in ../results/ENU_variants/long_flanks/sex_chroms/directions/CtoT/summary.txt are not significant
bonferroni corrected table ../results/ENU_variants/long_flanks/sex_chroms/d

A sequential Bonferroni correction involves dividing the significance level (0.05) by the total number of hypothesis tests. If the most "significant" test passes that criteria, we remove it and repeat the process.

We want another cogent Table that has the following columns:

* direction
* Position (see the summary.txt tables)
* prob

From these we will produce synopses of broad patterns, e.g. significant interactions are always between adjacent positions in sequence.

In [2]:
from cogent import LoadTable
##constructed a testing sample tbale with known probs and seq bonf sigc
header = ['Contrasts', 'prob']
rows = [['Sugars vs. control (C)(G,F,F + F,S)', 1e-6],
        ['Mixed vs. pure sugars (G + F)(G,F,S)', 0.004765],
        ['Among pure sugars (G)(F)(S)', 0.000002],
        ['Mixed ns. average of G and F (G+F)(G,F)', 0.418749],
        ['Monosaccharides vs. disaccharides (G,F)(S)', 1e-6]]

table = LoadTable(header = header, rows = rows)
print(table)
table.writeToFile('test_bonferroni.txt', sep='\t')

                                 Contrasts      prob
----------------------------------------------------
       Sugars vs. control (C)(G,F,F + F,S)    0.0000
      Mixed vs. pure sugars (G + F)(G,F,S)    0.0048
               Among pure sugars (G)(F)(S)    0.0000
   Mixed ns. average of G and F (G+F)(G,F)    0.4187
Monosaccharides vs. disaccharides (G,F)(S)    0.0000
----------------------------------------------------
